In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

def get_imgurl(soup):
    try:
        imgurl = soup.find("img", attrs={'id':'landingImage'}).get('src')
    except AttributeError:
        imgurl = ""
    return imgurl

def fetch_amazon_data(pages):
    HEADERS = ({'User-Agent': '', 'Accept-Language': 'en-US, en;q=0.5'})
    # just change the k = in below link for example for washing machine , just modify k = washing+machine so the modified link becomes https://www.amazon.com/s?k=washing+machine&ref=nb_sb_noss_2
    base_url = "https://www.amazon.com/s?k=table&ref=nb_sb_noss_2"
    links_list = []

    for page in range(1, pages + 1):
        URL = f"{base_url}&page={page}"
        webpage = requests.get(URL, headers=HEADERS)
        soup = BeautifulSoup(webpage.content, "html.parser")
        links = soup.find_all("a", attrs={'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

        for link in links:
            links_list.append(link.get('href'))

    d = {"title": [], "linkn": [], "imgurl": []}

    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        d['title'].append(get_title(new_soup))
        d['linkn'].append("https://www.amazon.com" + link)
        d['imgurl'].append(get_imgurl(new_soup))

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    # just name the file according to your category name i.e washingmachine.csv
    amazon_df.to_csv("table.csv", header=True, index=False)
    return amazon_df

if __name__ == '__main__':
    pages_to_scrape = 10  # Set the number of pages you want to scrape
    p = fetch_amazon_data(pages_to_scrape)


In [ ]:
# code for automation by category
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": 'productTitle'})
        title_value = title.text.strip()
    except AttributeError:
        title_value = ""
    return title_value

def get_imgurl(soup):
    try:
        imgurl = soup.find("img", attrs={'id': 'landingImage'}).get('src')
    except AttributeError:
        imgurl = ""
    return imgurl

def fetch_amazon_data(category):
    HEADERS = {'User-Agent': '', 'Accept-Language': 'en-US, en;q=0.5'}
    base_url = f"https://www.amazon.com/s?k={category}&ref=nb_sb_noss_2"
    links_list = []

    for page in range(1, pages_to_scrape + 1):
        URL = f"{base_url}&page={page}"
        webpage = requests.get(URL, headers=HEADERS)
        soup = BeautifulSoup(webpage.content, "html.parser")
        links = soup.find_all("a", class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')

        for link in links:
            links_list.append(link.get('href'))

    data = {"title": [], "linkn": [], "imgurl": [], "category": []}

    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        data['title'].append(get_title(new_soup))
        data['linkn'].append("https://www.amazon.com" + link)
        data['imgurl'].append(get_imgurl(new_soup))
        data['category'].append(category)

    amazon_df = pd.DataFrame(data)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df.dropna(subset=['title'], inplace=True)
    amazon_df.to_csv(f"{category}.csv", header=True, index=False)
    
    return amazon_df

if __name__ == '__main__':
    categories = ["watch", "jacket"]  # List of categories to scrape
    pages_to_scrape = 10  # Number of pages to scrape per category
    
    for category in categories:
        print(f"Scraping data for category: {category}")
        fetch_amazon_data(category)
        print(f"Scraping for category {category} complete.")
